# Debit Account Suggester Model Training

This notebook focuses on building and testing a debit account suggester using only the provided transaction data file: `Sheet 1-Table 1 Snapshot-1 Snapshot.csv`.

**Outline:**
1. Import Required Libraries
2. Load and Inspect the Active Document
3. Extract Debit Account Data
4. Implement Debit Account Suggestion Logic
5. Test Debit Account Suggestion Function

In [1]:
# 1. Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
# 2. Load and Inspect the Active Document
csv_path = '../flask_app/data/Sheet 1-Table 1 Snapshot-1 Snapshot.csv'
df = pd.read_csv(csv_path)
df.head()

,Date,Debit Account,Debit Amount,Credit Account,Credit Amount,Description
0,2024/5/1,KR10101-Toss Checking Account,328,KR70002-Interest Income,328,Interest Received
1,2024/5/2,KR60005-Personal Expenses,17500,KR10101-Toss Checking Account,17500,Myothurin
2,2024/5/2,KR60002-Food Expenses,9400,KR10101-Toss Checking Account,9400,Foreign Mart
3,2024/5/3,KR60005-Personal Expenses,8900,KR10101-Toss Checking Account,8900,iCloud+
4,2024/5/3,KR60002-Food Expenses,30000,KR10101-Toss Checking Account,30000,양꼬치 w 서진


In [3]:
# 3. Extract Debit Account Data
# We'll use Description, Debit Amount, and encode Debit Account for training
features = ['Description', 'Debit Amount']
target = 'Debit Account'

# Fill missing values
for col in features + [target]:
    df[col] = df[col].fillna('')

# Encode target labels
le = LabelEncoder()
df['debit_account_encoded'] = le.fit_transform(df[target])

# Vectorize description
tfidf = TfidfVectorizer(max_features=100)
desc_tfidf = tfidf.fit_transform(df['Description']).toarray()

# Combine features
X = np.hstack([desc_tfidf, df[['Debit Amount']].astype(float).values])
y = df['debit_account_encoded'].values

In [4]:
# 4. Implement Debit Account Suggestion Logic
# Train RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save model and encoders
joblib.dump(clf, '../flask_app/debit_account_suggester.joblib')
joblib.dump(le, '../flask_app/debit_account_label_encoder.joblib')
joblib.dump(tfidf, '../flask_app/debit_account_tfidf.joblib')

['../flask_app/debit_account_tfidf.joblib']

In [5]:
# 5. Test Debit Account Suggestion Function
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Example prediction function

def suggest_debit_account(description, amount):
    desc_vec = tfidf.transform([description]).toarray()
    amt_vec = np.array([[float(amount)]])
    X_input = np.hstack([desc_vec, amt_vec])
    pred = clf.predict(X_input)
    return le.inverse_transform(pred)[0]

# Test example
print(suggest_debit_account('Pasta w Alua', 22400))

Test Accuracy: 0.7143


ValueError: Number of classes, 23, does not match size of target_names, 29. Try specifying the labels parameter